# Cálculo de otros indicadores simples

En este notebook se calculan indicadores sencillos varios.

In [6]:
from scripts.green_land import calculate_green_spaces
from scripts.count_points import count_points
from scripts.path_finders import get_division_path, get_context_shp_path, get_transport_shp_path
from scripts.path_finders import  get_area_level_shp_path
from scripts.create_indicators import add_indicator, get_or_create_indicators_df, add_dbf_indicator_by_id
import shapefile
from simpledbf import Dbf5
from shapely.geometry import MultiPoint
from scripts.geo_utils import iter_shp_as_shapely

In [7]:
radio_shp = get_division_path("radios_censo_2010")
fracc_shp = get_division_path("fracciones_caba_censo_2010")
#barrio_shp = get_division_path("barrios_censo_2010")
comuna_shp = get_division_path("comunas_caba_censo_2010")

## 1. Incidencia de espacios verdes

Calculamos la incidencia de espacios verdes en cada radio censal.

In [8]:
green_privados_shp = get_context_shp_path("espacios-verdes-privados")
green_publicos_shp = get_context_shp_path("espacios-verdes-publicos")

In [9]:
# puede tardar bastante
radios_green = calculate_green_spaces(radio_shp, [green_privados_shp, green_publicos_shp])

In [10]:
radios_green["1_13_14"]

Agregamos el indicador a la base

In [11]:
df = add_indicator("RADIO", "esp_verde", radios_green)

## 2. Cantidad de "puntos de interés"

Contamos la cantidad de hospitales y establecimientos educativos por cada radio censal.

In [12]:
hospitales_shp = get_context_shp_path("hospitales")
educ_privados_shp = get_context_shp_path("Establecimientos Educativos Privados")
educ_publicos_shp = get_context_shp_path("Establecimientos Educativos Publicos")

In [14]:
hospitales = count_points(hospitales_shp, radio_shp)
print sum(hospitales.values()), "hospitales contados"

36 hospitales contados


Agregamos los indicadores a la base

In [17]:
def add_count_points_indicator(area_levels, indic_names, context_shps):
    for area_level in area_levels:
        for indic_name, context_shp in zip(indic_names, context_shps):
            print "Counting", indic_name.ljust(10), "in", area_level
            shp_path = get_area_level_shp_path(area_level)
            df = add_indicator(area_level, indic_name, count_points(context_shp, shp_path))

In [18]:
add_count_points_indicator(["RADIO", "FRAC", "DPTO"], ["hospitales", "educ_priv", "educ_pub"],
                           [hospitales_shp, educ_privados_shp, educ_publicos_shp])

Counting hospitales in RADIO
Counting educ_priv  in RADIO
Counting educ_pub   in RADIO
Counting hospitales in FRAC
Counting educ_priv  in FRAC
Counting educ_pub   in FRAC
Counting hospitales in DPTO
Counting educ_priv  in DPTO
Counting educ_pub   in DPTO


## 3. Distancia media de los habs de un radio censal a puntos de transporte

In [19]:
def calculate_distance_indicators(df_indicators, id_field, division_shp_name, indicators):
    shapes_dict = {shape[0]: shape[1] for shape 
                   in iter_shp_as_shapely(get_division_path(division_shp_name))}
    
    for indic_name, indic_shp_name in indicators.iteritems():
        print "Calculating", indic_name, indic_shp_name
        
        multipoint = MultiPoint([shape[1].centroid for shape in 
                                 iter_shp_as_shapely(get_transport_shp_path(indic_shp_name))])

        def get_distance(id_shape):
            return shapes_dict[id_shape].centroid.distance(multipoint)
        
        df_indicators[indic_name] = map(get_distance, df_indicators[id_field])    

In [20]:
indicadores = {"d_subte": "subte-estaciones",
               "d_ffcc": "estaciones-ferrocarril",
               "d_metrobus": "metrobus-estaciones"}
indicators_radio = get_or_create_indicators_df("RADIO")
calculate_distance_indicators(indicators_radio, "CO_FRAC_RA", "radios_censo_2010", indicadores)

Calculating d_metrobus metrobus-estaciones
Calculating d_ffcc estaciones-ferrocarril
Calculating d_subte subte-estaciones


## 4. Incorporación de indicadores precalculados en shapefiles

In [21]:
df = add_dbf_indicator_by_id("RADIO", "CABA_INSE", "LINK", "DECIL")